## Exploring the Data

In [1]:
import os
import polars as pl
from utils import *

Let's read the data corresponding to a single state ('AC' in this case) to explore it.

In [2]:
csv_files_dir = '../data/csv_files'
file = 'AC.csv'
file_path = os.path.join(csv_files_dir, file)
df = read_csv(file_path, separator = ',')

Let's check the different types of 'ClassInfraFisica' present.

In [3]:
df.group_by('ClassInfraFisica').count()

ClassInfraFisica,count
str,u32
"""Rooftop""",129
"""Greenfield""",2535
null,6157
"""Indoor""",7
"""Ran Sharing""",39
"""Streetlevel """,9
"""Streetlevel""",15


Let's fix the typo:

In [4]:
df = replace_values(df, 'ClassInfraFisica', 'Streetlevel ', 'Streetlevel')

In [5]:
df.group_by('ClassInfraFisica').agg(pl.col('ClassInfraFisica').count().alias('Count'))

ClassInfraFisica,Count
str,u32
null,6157
"""Ran Sharing""",39
"""Greenfield""",2535
"""Rooftop""",129
"""Indoor""",7
"""Streetlevel""",24


Now, we have to check whether each station (indicated by the column 'NumEstacao') corresponds to a single type of 'ClassInfraFisica'; which is the expected behavior (assumption).

In [6]:
count_unique_values = df.group_by('NumEstacao').agg(pl.col('ClassInfraFisica').n_unique().alias('unique_count'))
instances_with_different_values = count_unique_values.filter(count_unique_values['unique_count'] > 1)
print(instances_with_different_values)

shape: (163, 2)
┌────────────┬──────────────┐
│ NumEstacao ┆ unique_count │
│ ---        ┆ ---          │
│ i64        ┆ u32          │
╞════════════╪══════════════╡
│ 692777830  ┆ 2            │
│ 1008864215 ┆ 2            │
│ 1064380    ┆ 2            │
│ 1002372221 ┆ 2            │
│ …          ┆ …            │
│ 1006699802 ┆ 2            │
│ 1002045166 ┆ 2            │
│ 1001799248 ┆ 2            │
│ 1007652486 ┆ 2            │
└────────────┴──────────────┘


We found that we can actually get two different 'ClassInfraFisica' for any individual station. But if we check further, we'll see that it's actually pairing types with 'null' values, which does not compromise the assumption.

In [7]:
df.filter(df['NumEstacao'] == 699785804)[['NumEstacao', 'ClassInfraFisica']]

NumEstacao,ClassInfraFisica
i64,str
699785804,null
699785804,null
699785804,null
699785804,"""Greenfield"""
699785804,"""Greenfield"""
699785804,"""Greenfield"""
699785804,"""Greenfield"""
699785804,"""Greenfield"""
699785804,"""Greenfield"""


That's as far as I can go with polars. Let's move to pandas.

In [8]:
df = df.to_pandas()          

Let's group by 'NumEstacao', but keeping all the info present in the rows as sets (or as a single value if all rows match)

In [9]:
def set_aggregation(x):

    if len(set(x)) == 1:
        return x.iloc[0]  
    else:
        return set(x)  

dfg = df.groupby('NumEstacao').agg(lambda x: set_aggregation(x))
dfg.head(4)

,Status.state,NomeEntidade,NumFistel,NumServico,NumAto,EnderecoEstacao,EndComplemento,SiglaUf,CodMunicipio,DesignacaoEmissao,...,Latitude,Longitude,CodDebitoTFI,DataLicenciamento,DataPrimeiroLicenciamento,NumRede,_id,DataValidade,NumFistelAssociado,NomeEntidadeAssociado
NumEstacao,,,,,,,,,,,,,,,,,,,,,
19453,LIC-LIC-01,EMPRESA BRASILEIRA DE INFRA-ESTRUTURA AEROPORT...,11030016470,19,655742007.0,"BR 364, KM 18 - COA s/n AEROPORTO SBRB",None,AC,1200401,16K0F3E,...,-9.9925,-67.804167,A,2016-03-03,2001-10-09,705,"{4d469248e6c2d201, 4d469248e6c2d1ff, 4d469248e...",2027-07-16,None,None
19461,LIC-LIC-01,EMPRESA BRASILEIRA DE INFRA-ESTRUTURA AEROPORT...,11030016470,19,18652020.0,"BR 364, KM 18 - COE s/n AEROPORTO SBRB",None,AC,1200401,16K0F3E,...,-9.992778,-67.804444,A,2016-03-03,2001-10-09,705,"{4d469248e6c2d208, 4d469248e6c2d210, 4d469248e...",2027-07-16,None,None
19470,LIC-LIC-01,EMPRESA BRASILEIRA DE INFRA-ESTRUTURA AEROPORT...,11030016470,19,18652020.0,"BR 364, KM 18 - SCI s/n AEROPORTO SBRB",None,AC,1200401,16K0F3E,...,-9.992778,-67.804722,A,2016-03-03,2001-10-09,705,"{4d469248e6c2d211, 4d469248e6c2d212, 4d469248e...",2027-07-16,None,None
64300,LIC-LIC-01,TELEFONICA BRASIL S.A.,50409146366,10,"{105802021.0, 48422023.0, 59072012.0, 97432014...","RUA FLORIANO PEIXOTO, 358",None,AC,1200401,"{5M00G7W, 5M00G9W, 200KG7W, 100MG7W, 10M0G7W, ...",...,-9.972025,-67.813556,G,2023-08-05,2000-11-27,None,"{5afddca4638ab, 4d5c019f58c85, aadc95f8f7a88cd...",2024-07-21,None,None


In [10]:
print(f"Percentage of initial rows kept when grouping = {len(dfg)*100/len(df):.2f}%")

Percentage of initial rows kept when grouping = 16.94%


Now we have a dataframe with all the original information, but only 16% of the rows; in which each represents a single station

## Understanding the Columns

Let's understand what the columns are about. This will futurely enable us to better make sense of them, process them if necessary, or drop irrelevant ones.

In [11]:
dfg.columns

Index(['Status.state', 'NomeEntidade', 'NumFistel', 'NumServico', 'NumAto',
       'EnderecoEstacao', 'EndComplemento', 'SiglaUf', 'CodMunicipio',
       'DesignacaoEmissao', 'Tecnologia', 'tipoTecnologia', 'meioAcesso',
       'FreqTxMHz', 'FreqRxMHz', 'Azimute', 'CodTipoClasseEstacao',
       'ClassInfraFisica', 'CompartilhamentoInfraFisica', 'CodTipoAntena',
       'CodEquipamentoAntena', 'GanhoAntena', 'FrenteCostaAntena',
       'AnguloMeiaPotenciaAntena', 'AnguloElevacao', 'Polarizacao',
       'AlturaAntena', 'CodEquipamentoTransmissor', 'PotenciaTransmissorWatts',
       'Latitude', 'Longitude', 'CodDebitoTFI', 'DataLicenciamento',
       'DataPrimeiroLicenciamento', 'NumRede', '_id', 'DataValidade',
       'NumFistelAssociado', 'NomeEntidadeAssociado'],
      dtype='object')

From [Página de Emissões - Anatel](https://sistemas.anatel.gov.br/anexar-api/publico/anexos/download/9cfc11fc83fcfc2d2586cdb887f72cb5), we can understand what each column refers to.

Status.state: Situação da solicitação na Anatel.

NumFistel: É um código numérico definido pela Anatel, composto de onze dígitos que identifica a autorização que a empresa possui para determinado serviço em determinada região.

NumServico: Código do serviço de telecomunicações na Anatel.

NumAto: Número do Ato de Autorização de Uso de Radiofrequência, sendo que os quatro últimos dígitos se referem ao ano da assinatura do respectivo ato. Exemplo XYZW2017, ato XYZW de 2017.

EnderecoEstacao: Endereço completo onde a estação está instalada.

EndComplemento: Complemento do endereço, caso haja.

DesignacaoEmissao: Baseado em [Método de Designação - Anatel](https://www.anatel.gov.br/Portal/verificaDocumentos/documento.asp?numeroPublicacao=60403&assuntoPublicacao=null&caminhoRel=null&filtro=1&documentoPath=outros/autocadastramento/metodo_de_designacao.pdf)

Designação de emissão. Para uma completa designação da emissão, necessitamos sempre de 9 caracteres alfanuméricos. Os quatros primeiros representam a _largura de faixa necessária_; os três seguintes as _características básicas_ e os dois últimos as _características adicionais facultativas_.
- Largura de faixa necessária: Para uma dada classe de emissão, o valor mínimo da largura de faixa ocupada pela emissão, suficiente para garantir a transmissão da informação com a velocidade de transmissão e com a qualidade requerida para o sistema empregado, nas condições especificadas. A largura de faixa necessária será sempre expressa por meio de três algarismos, que indicam os três primeiros algarismos significativos de largura de faixa necessária e uma letra que ocupa a posição da virgula decimal e representa a Unidade de largura de faixa e será H para Hertz, K para Kilohertz, M para Megahertz ou G para Gigahertz.
- Características básicas: descritas por três símbolos:
    - Primeiro símbolo: tipo de modulação da portaria principal.
    - Segundo símbolo: natureza do(s) sinais que modulam a portaria principal.
    - Terceiro símbolo: tipo de informação a ser transmitida.
- Características adicionais facultativas: Para descrição mais completa de uma emissão, são previstas duas características facultativas, as quais são expressas pelos quarto e quinto símbolos. Quando não se utiliza o quarto ou o quinto símbolo, convém indicar isso mediante um traço no lugar em que cada símbolo apareceria.

meioAcesso: é um campo de ‘estações dispensadas de licenciamento’. As opções são: fibra, par metálico, cabo coaxial e radiação restrita.

Azimute: posicionamento m graus em relação ao Norte do lóbulo principal de radiação da antena. Quando for utilizada antena omnidirecional será 0.

CodTipoClasseEstacao: Classe da Estação conforme lista permitida no sistema de canalização do Mosaico, conforme o Regulamento da faixa de frequência utilizada conforme Manual de Projetos Técnicos do SITAR, disponível em [Manual de Projeto - Anatel](https://www.anatel.gov.br/Portal/verificaDocumentos/documento.asp?numeroPublicacao=60402&assuntoPublicacao=MANUAL%20DE%20PROJETOS%20T%C9CNICOS%20(SITAR)&caminhoRel=CidadaoComunica%E7%E3o%20via%20R%E1dioServi%E7o%20Limitado&filtro=1&documentoPath=outros/). Exemplos:
- ML: Estação móvel terrestre
- FX: Estação fixa
- FB: Estação de base
- XR: Estação fixa repetidora
- FA: Estação aeronáutica
- RC: Radiofarol não direcional

ClassInfraFisica: Classificação de Infraestrutura Física. Apenas para estações dos serviços de interesse coletivo (STFC, SCM e SMP). Para serviços de interesse restrito (SLP), estará em branco. Valores válidos: Greenfield, Streetlevel, Ran Sharing, Small Cell, COW, Picocelula, Harmonizada, Rooftop, Indoor, Outdoor

CompartilhamentoInfraFisica: Compartilhamento de Infraestrutura Física. Apenas para estações dos serviços de interesse coletivo (não se aplica para SLP).

CodTipoAntena: Tipo Antena: campo obrigatório, onde é inserido o código conforme item 2.3.4 do [Manual de Projeto - Anatel](https://www.anatel.gov.br/Portal/verificaDocumentos/documento.asp?numeroPublicacao=60402&assuntoPublicacao=MANUAL%20DE%20PROJETOS%20T%C9CNICOS%20(SITAR)&caminhoRel=CidadaoComunica%E7%E3o%20via%20R%E1dioServi%E7o%20Limitado&filtro=1&documentoPath=outros/). Exemplos:
- 019: Monopolo vertical
- 060: V invertido
- 213: Cabo fendido

GanhoAntena: Ganho da Antena em dB. Se a frequência for superior a 28.000 kHz, é informado o ganho da antena em relação a uma antena isotrópica (dbi); se inferior, este é dado em relação a uma antena dipolo (dBd).

FrenteCostaAntena: Relação Frente/Costa em dBi (campo obrigatório). Deve ser menor que 90 dbi. Será 0 (zero) quando for utilizada antena omnidirecional.

AnguloMeiaPotenciaAntena: Ângulo de meia potência em graus decimais (campo obrigatório). Pontos no diagrama onde a potência radiada equivale à metade da radiada na direção principal.

In [18]:
# meioAcesso
dfg['meioAcesso'].value_counts(dropna=False)

meioAcesso
None    1506
Name: count, dtype: int64

In [20]:
dfg['CodTipoClasseEstacao'].value_counts()

CodTipoClasseEstacao
ML              719
FX              358
FB              337
{FB, FX}         62
BR               10
{BR, XR}          8
XR                4
FA                3
{XR, FX}          2
RC                2
{FP, FB, FC}      1
Name: count, dtype: int64